In [1]:
#https://github.com/clarketm/mergedeep/tree/master/mergedeep
from collections import Counter
from collections.abc import Mapping
from copy import deepcopy
from enum import Enum
from functools import reduce, partial
from typing import MutableMapping




class Strategy(Enum):
    # Replace `destination` item with one from `source` (default).
    REPLACE = 0
    # Combine `list`, `tuple`, `set`, or `Counter` types into one collection.
    ADDITIVE = 1
    # Alias to: `TYPESAFE_REPLACE`
    TYPESAFE = 2
    # Raise `TypeError` when `destination` and `source` types differ. Otherwise, perform a `REPLACE` merge.
    TYPESAFE_REPLACE = 3
    # Raise `TypeError` when `destination` and `source` types differ. Otherwise, perform a `ADDITIVE` merge.
    TYPESAFE_ADDITIVE = 4


def _handle_merge_replace(destination, source, key):
    if isinstance(destination[key], Counter) and isinstance(source[key], Counter):
        # Merge both destination and source `Counter` as if they were a standard dict.
        _deepmerge(destination[key], source[key])
    else:
        # If a key exists in both objects and the values are `different`, the value from the `source` object will be used.
        destination[key] = deepcopy(source[key])


def _handle_merge_additive(destination, source, key):
    # Values are combined into one long collection.
    if isinstance(destination[key], list) and isinstance(source[key], list):
        # Extend destination if both destination and source are `list` type.
        destination[key].extend(deepcopy(source[key]))
    elif isinstance(destination[key], set) and isinstance(source[key], set):
        # Update destination if both destination and source are `set` type.
        destination[key].update(deepcopy(source[key]))
    elif isinstance(destination[key], tuple) and isinstance(source[key], tuple):
        # Update destination if both destination and source are `tuple` type.
        destination[key] = destination[key] + deepcopy(source[key])
    elif isinstance(destination[key], Counter) and isinstance(source[key], Counter):
        # Update destination if both destination and source are `Counter` type.
        destination[key].update(deepcopy(source[key]))
    else:
        _handle_merge[Strategy.REPLACE](destination, source, key)


def _handle_merge_typesafe(destination, source, key, strategy):
    # Raise a TypeError if the destination and source types differ.
    if type(destination[key]) is not type(source[key]):
        raise TypeError(
            f'destination type: {type(destination[key])} differs from source type: {type(source[key])} for key: "{key}"'
        )
    else:
        _handle_merge[strategy](destination, source, key)


_handle_merge = {
    Strategy.REPLACE: _handle_merge_replace,
    Strategy.ADDITIVE: _handle_merge_additive,
    Strategy.TYPESAFE: partial(_handle_merge_typesafe, strategy=Strategy.REPLACE),
    Strategy.TYPESAFE_REPLACE: partial(_handle_merge_typesafe, strategy=Strategy.REPLACE),
    Strategy.TYPESAFE_ADDITIVE: partial(_handle_merge_typesafe, strategy=Strategy.ADDITIVE),
}


def _is_recursive_merge(a, b):
    both_mapping = isinstance(a, Mapping) and isinstance(b, Mapping)
    both_counter = isinstance(a, Counter) and isinstance(b, Counter)
    return both_mapping and not both_counter


def _deepmerge(dst, src, strategy=Strategy.REPLACE):
    for key in src:
        if key in dst:
            if _is_recursive_merge(dst[key], src[key]):
                # If the key for both `dst` and `src` are both Mapping types (e.g. dict), then recurse.
                _deepmerge(dst[key], src[key], strategy)
            elif dst[key] is src[key]:
                # If a key exists in both objects and the values are `same`, the value from the `dst` object will be used.
                pass
            else:
                _handle_merge.get(strategy)(dst, src, key)
        else:
            # If the key exists only in `src`, the value from the `src` object will be used.
            dst[key] = deepcopy(src[key])
    return dst


def merge(destination: MutableMapping, *sources: Mapping, strategy: Strategy = Strategy.REPLACE) -> MutableMapping:
    """
    A deep merge function for 🐍.

    :param destination: The destination mapping.
    :param sources: The source mappings.
    :param strategy: The merge strategy.
    :return:
    """
    return reduce(partial(_deepmerge, strategy=strategy), sources, destination)

def split(data, value=2):
    """recursively turn a set of words into a nested dict
    https://stackoverflow.com/questions/52349884/create-a-nested-dictionary-using-recursion-python
    """
    if data:
        head, *tail = data  # This is a nicer way of doing head, tail = data[0], data[1:]
        return {head: split(tail,value)}
    else:
        return value  
        # this returns the value to the previous split() 
        #    which sets it in the line above by calling itself and returns the datastructure
        #    now that data is None, the complete dataStructure is returned back up through call stack
        #    to the original call.  
        # It's way over my head!
    


def fixV(s):
    """
    deal with non-string values
    """
    if s in ['True', 'true']: return True
    if s in ['False', 'false']: return False
    try: # to convert to a number
        return(int(s))
    except:
        return s #then just return as is

def makeOpt(line = 'nodes color background red'):
    """
    prepare string for split
    """
    words = line.split()
    value = fixV(words.pop())
        
    return split(words, value)


d1 = makeOpt('nodes color background red')
d2 = makeOpt('nodes color border green')
d3 = makeOpt('nodes shape box')
options = merge(d1,d2,d3)
print(options)

options = merge(makeOpt('nodes borderWidth 12'), options)
print(options)



In [60]:
line='node color background red'
d=dict()
words=line.split()
value=words.pop()
firstKey=words.pop(0)
while words:
    d[firstKey][word]=value

'node'

In [71]:
nd = nested_dict()
nd['node']['color']='red'
try:
    nd['node']['color']['background'] = 'blue'
except TypeError:
    print('fixed')
    nd['node']['color']=dict()
    nd['node']['color']['background']='blue'
nd.to_dict()

fixed


{'node': {'color': {'background': 'blue'}}}

In [161]:
def setValue(d={}, words='node color background red'):
    """this breask if you have to go up two levels (ok?)"""
    words=words.split(' ')
    nd=nested_dict(d)
    value = words.pop()
    if len(words)==1: #easy
        print(1)
        nd[words[0]] = value
    if len(words)==2: 
        print(2)
        nd[words[0]]=nested_dict() #if words[0] is defined we need to undefine it 
        nd[words[0]][words[1]] = value
    if len(words)==3:
        print(3)
        #nd[words[0]]={}
        nd[words[0]][words[1]]=nested_dict()
        nd[words[0]][words[1]][words[2]] = value
    if len(words)==4:
        print(4)
        #nd[words[0]]={}
        #nd[words[0]][words[1]]={}
        nd[words[0]][words[1]][words[2]] = nested_dict()
        nd[words[0]][words[1]][words[2]][words[3]]=value

    return nd.to_dict()
d={}
d = setValue(d,'color red')
d
d = setValue(d,'color color background blue also')
d
d = setValue(d,'nodes color border blue also')
d

1
4


TypeError: string indices must be integers

In [172]:
words='one two three four five'.split()
d=nested_dict()
d?

Type:           nested_dict
String form:    {}
Length:         0
Docstring:     
Nested dict.

Uses defaultdict to automatically add levels of nested dicts and other types.
Init docstring:
Constructor.

Takes one or two parameters
    1) int, [TYPE]
    1) dict


In [26]:
nd.to_dict().items()

dict_items([(1, {2: {3: 4}}), (4, {5: {6: 7}})])

In [269]:
def split(data, value=2):
    """recursively turn a set of words into a nested dict
    https://stackoverflow.com/questions/52349884/create-a-nested-dictionary-using-recursion-python
    """
    if data:
        head, *tail = data  # This is a nicer way of doing head, tail = data[0], data[1:]
        return {head: split(tail,value)}
    else:
        return value  
        # this returns the value to the previous split() 
        #    which sets it in the line above by calling itself and returns the datastructure
        #    now that data is None, the complete dataStructure is returned back up through call stack
        #    to the original call.  
        # It's way over my head!
    
d= split('nodes color background red'.split(), value = 'red')
d2= split('nodes color border green'.split(), value = 'green')
d,d2

({'nodes': {'color': {'background': {'red': 'red'}}}},
 {'nodes': {'color': {'border': {'green': 'green'}}}})

In [4]:
#https://github.com/clarketm/mergedeep/tree/master/mergedeep
from collections import Counter
from collections.abc import Mapping
from copy import deepcopy
from enum import Enum
from functools import reduce, partial
from typing import MutableMapping




class Strategy(Enum):
    # Replace `destination` item with one from `source` (default).
    REPLACE = 0
    # Combine `list`, `tuple`, `set`, or `Counter` types into one collection.
    ADDITIVE = 1
    # Alias to: `TYPESAFE_REPLACE`
    TYPESAFE = 2
    # Raise `TypeError` when `destination` and `source` types differ. Otherwise, perform a `REPLACE` merge.
    TYPESAFE_REPLACE = 3
    # Raise `TypeError` when `destination` and `source` types differ. Otherwise, perform a `ADDITIVE` merge.
    TYPESAFE_ADDITIVE = 4


def _handle_merge_replace(destination, source, key):
    if isinstance(destination[key], Counter) and isinstance(source[key], Counter):
        # Merge both destination and source `Counter` as if they were a standard dict.
        _deepmerge(destination[key], source[key])
    else:
        # If a key exists in both objects and the values are `different`, the value from the `source` object will be used.
        destination[key] = deepcopy(source[key])


def _handle_merge_additive(destination, source, key):
    # Values are combined into one long collection.
    if isinstance(destination[key], list) and isinstance(source[key], list):
        # Extend destination if both destination and source are `list` type.
        destination[key].extend(deepcopy(source[key]))
    elif isinstance(destination[key], set) and isinstance(source[key], set):
        # Update destination if both destination and source are `set` type.
        destination[key].update(deepcopy(source[key]))
    elif isinstance(destination[key], tuple) and isinstance(source[key], tuple):
        # Update destination if both destination and source are `tuple` type.
        destination[key] = destination[key] + deepcopy(source[key])
    elif isinstance(destination[key], Counter) and isinstance(source[key], Counter):
        # Update destination if both destination and source are `Counter` type.
        destination[key].update(deepcopy(source[key]))
    else:
        _handle_merge[Strategy.REPLACE](destination, source, key)


def _handle_merge_typesafe(destination, source, key, strategy):
    # Raise a TypeError if the destination and source types differ.
    if type(destination[key]) is not type(source[key]):
        raise TypeError(
            f'destination type: {type(destination[key])} differs from source type: {type(source[key])} for key: "{key}"'
        )
    else:
        _handle_merge[strategy](destination, source, key)


_handle_merge = {
    Strategy.REPLACE: _handle_merge_replace,
    Strategy.ADDITIVE: _handle_merge_additive,
    Strategy.TYPESAFE: partial(_handle_merge_typesafe, strategy=Strategy.REPLACE),
    Strategy.TYPESAFE_REPLACE: partial(_handle_merge_typesafe, strategy=Strategy.REPLACE),
    Strategy.TYPESAFE_ADDITIVE: partial(_handle_merge_typesafe, strategy=Strategy.ADDITIVE),
}


def _is_recursive_merge(a, b):
    both_mapping = isinstance(a, Mapping) and isinstance(b, Mapping)
    both_counter = isinstance(a, Counter) and isinstance(b, Counter)
    return both_mapping and not both_counter


def _deepmerge(dst, src, strategy=Strategy.REPLACE):
    for key in src:
        if key in dst:
            if _is_recursive_merge(dst[key], src[key]):
                # If the key for both `dst` and `src` are both Mapping types (e.g. dict), then recurse.
                _deepmerge(dst[key], src[key], strategy)
            elif dst[key] is src[key]:
                # If a key exists in both objects and the values are `same`, the value from the `dst` object will be used.
                pass
            else:
                _handle_merge.get(strategy)(dst, src, key)
        else:
            # If the key exists only in `src`, the value from the `src` object will be used.
            dst[key] = deepcopy(src[key])
    return dst


def merge(destination: MutableMapping, *sources: Mapping, strategy: Strategy = Strategy.REPLACE) -> MutableMapping:
    """
    A deep merge function for 🐍.

    :param destination: The destination mapping.
    :param sources: The source mappings.
    :param strategy: The merge strategy.
    :return:
    """
    return reduce(partial(_deepmerge, strategy=strategy), sources, destination)



In [5]:
def fixV(s):
    """
    deal with non-string values
    """
    if s in ['True', 'true']: return True
    if s in ['False', 'false']: return False
    try: # to convert to a number
        return(int(s))
    except:
        return s #then just return as is

def makeOpt(line = 'nodes color background red'):
    """
    prepare string for split
    """
    words = line.split()
    value = fixV(words.pop())
        
    return split(words, value)


d1 = makeOpt('nodes color background red')
d2 = makeOpt('nodes color border green')
d3 = makeOpt('nodes shape box')
options = merge(d1,d2,d3)
options

NameError: name 'split' is not defined

In [6]:
options = merge(makeOpt('nodes borderWidth 12'), options)
options

NameError: name 'split' is not defined